# Chat Agent Executor
In this example we will build a chat executor that uses function calling from scratch.

### **Setup**
First we need to install the packages required.

In [1]:
%pip install --quiet -U langchain langchain_openai tavily-python langchain_community langgraph dotenv

Note: you may need to restart the kernel to use updated packages.


Next, we need to set API keys for OpenAI (the LLM we will use) and Tavily (the search tool we will use)

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] =  getpass.getpass("OpenAI Api Key:")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API Key:")

Optionally, we can set API key for LangSmith tracing, which will give us best-in-class observability.

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key:")

### **Set up the tools**

We will first define the tools we want to use. For this simple example, we will use a built-in search tool via Tavily. However, it is really easy to create your own tools- documentation [here]('https://python.langchain.com/docs/how_to/custom_tools/') on how to do that.

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

We can now wrap these tools in a simple ToolExecutor. This is a real simple class that takes in a ToolInvocation and calls that tool, returning the output. A ToolInvocation is any class with tool and tool_input attribute.

At the time of creation of this notebook, 'ToolExecutor' has been depreciated. Use [ToolNode]('https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.tool_node.ToolNode') instead.

In [60]:
from langgraph.prebuilt import ToolExecutor # ToolExecutor has been depreciated. 

tool_executor = ToolExecutor(tools)

/var/folders/j9/7txjkxjn6zs87g_3bm9f81fr0000gn/T/ipykernel_53483/1172846420.py:3: LangGraphDeprecationWarning: ToolExecutor is deprecated as of version 0.2.0 and will be removed in 0.3.0. Use langgraph.prebuilt.ToolNode instead.
  tool_executor = ToolExecutor(tools)


### **Set up the model**

Now we need to load the chat model we want to use. Importantly, this should satisfy two criteria:

1. It should work with messages. We will represent all agent state in the form of messages, so it needs to be able to work well with them.
2. It should work with OpenAi function calling. This means it should either be an OpenAI model or a model that exposes a similar interface.

Note: these model requirements are not requirements for using LangGraph - they are just requirements for this one example.

In [61]:
from langchain_openai import ChatOpenAI

# We will set streaming=True so that we can stream tokens
# See the streaming section for more information on this.
model = ChatOpenAI(temperature=0, streaming=True)

After we've done this, we should make sure the model knows that it has these tools available to call. We can do this by converting the LangChain tools into the format of OpenAi function calling, and then bind them to the model class.

In [85]:
from langchain_core.utils.function_calling import format_tool_to_openai_function

functions = [format_tool_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

### **Define the agent state**

The main type of graph in langgraph is the StatefulGraph. This graph is parameterized by a state object that it passes around to each node. Each node then returns operations to update that state. These operations can either SET specific attributes on the state (e.g overwrite the existing values) or ADD to the existing attribute. Whether to set or add is denoted by annotating the state object you construct the graph with.

For this example, the state we will track will just be a list of messages. We want each node to just add messages to that list. Therefore, we will use a TypedDict with one key (messages) and annotate it so that the messages attribute is always added to.



In [86]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

### **Define the nodes**

We now need to define a few different nodes in our graph. In langgraph, a node can be either a function or a runnable. There are two main nodes we need for this:

1. The agent: responsible for deciding what (if any) actions to take.
2. A function to invoke tools: if the agent decides to take an action, this node will then execute that action.

We will also need to define the edges. Some of these edges may be conditional. The reason they are conditional is that based on the output of a node, one of several paths may be taken. The path that is taken is not knwon until that node is run (the LLM decides).

1. Conditional Edge: after the agent is called, we should either:

    a. if the agent sad to take an action, then the function to invoke tools should be called

    b. if the agent said that it was finished, then it should finish

2. Normal Edge: after the tools are invoked, it should always go back to the agent to decide what to do next

Let's define the noedes, as well as a function to decide how what conditional edge to take.


In [87]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

# Define the funciton that determiens whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "tool_calls" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# Define the funciton that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added ot the existing list
    return {"messages": [response]}

# Define the function to execute tools
def call_tools(state):
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct a ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["tool_calls"][0]["function"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["tool_calls"][0]["function"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to creat a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message] }

### **Define the graph**

We can now put it all together and define the graph!


In [88]:
from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tools)

# Set the entrypoint as 'agent'
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the state node. We use 'agent',
    # This means these are the edges taken after the 'agent' node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping,
    # The keys are strings, an the values are other nodes,
    # END is a special node marking that the graph should finish,
    # What will happen is we will call 'should_continue', and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node wil then be called.
    {
        # If 'tools', then we call the tool node.
        "continue": "action",
        # Otherwise we finish,
        "end": END
    }
)

# We now add a normal edge from 'tools' to 'agent'.
# This means that after 'tools' is called, 'agent' node is called next
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable
# meaning you can use it as you would any other runnable
app = workflow.compile()

### **Use it!**

We can now use it! This now exposes the same interface as all other LangChain runnables.

In [89]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in frisco, tx")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in frisco, tx', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in Frisco, TX"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-1772ef6e-23cc-4446-94fa-62d4e51f8f07-0')]}